Proyecto opcional, sistema de encriptacion y decriptacion con la contraseña que el usuario desee.

In [ ]:
!pip install cryptography
!pip install ipywidgets

In [ ]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import os
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Funcion para derivar la llave
def generate_key(password, salt=None): #Definimos "generate_key" como funcion
    if salt is None:                   #con los argumentos "password" &"salt"
        salt = os.urandom(16)          #"salt" siendo un random value generator
        
    kdf = PBKDF2HMAC(               #Usamos PBKDF2HMAC (Password-Based Key Derivation Function 2 with HMAC) para kdf
        algorithm=hashes.SHA256(),  #Especificamos que usamos el algoritmo de hashing: SHA256 
        length=32,                  #Va a ser un 32-byte key 
        salt=salt,                  
        iterations=100000,          #Usamos 100mil iteraciones, mientras mas iteraciones, mas dificil un ataque en la llave derivada.
    )
    
    return base64.urlsafe_b64encode(kdf.derive(password.encode())), salt

    #Este ultimo return hace varios pasos, encodea la contraseña como bytes
    #Una vez encodeado, se llama al derive de kdf
    #base64.urlsafe_encode lo uso porque daba un error al poner texto como
    #contraseña en vez de 1's y 0's, esto hace que sea posible usarse en url's 
    #y otros text formats.



In [ ]:
# Funcion de encriptado
def encrypt(message, key):
    f = Fernet(key)
    return f.encrypt(message.encode())

# message es el dato o mensaje que quiero encriptar
# key es la llave simetrica de encripcion que se va a usar para el mensaje
# al poner el key en la funcion fernet se encripta el mensaje en 128 bits


In [ ]:
# Funcion d edecriptado
def decrypt(token, key):
    f = Fernet(key)
    return f.decrypt(token).decode()

#Mismo proceso que el encrypt function solo que revertido.
#llama el decrypt method del fernet con la llave proveida.
#decodea el mensaje encriptado en bits de vuelta a string.
#y devuelve el mensaje decriptado como string.

In [ ]:
import base64

def on_encrypt_button_click(button):
    password = encrypt_password_input.value
    message = message_input.value

    # Generamos una llave usando la contraseña.
    key, salt = generate_key(password)

    # Encriptamos el mensaje
    encrypted_message = encrypt(message, key)
    print(f"Encrypted message: {encrypted_message}")

    # Almacenamos el mensaje cifrado y la sal en el widget para su desencriptación.
    encrypted_message_output.value = encrypted_message
    salt_output.value = base64.b64encode(salt).decode()

def on_decrypt_button_click(button):
    password = decrypt_password_input.value

    # Recuperamos el mensaje cifrado y la sal desde el widget.
    encrypted_message = encrypted_message_output.value
    salt = base64.b64decode(salt_output.value.encode())

    if encrypted_message and salt:
        # Generamos una clave usando la contraseña y salt.
        key, _ = generate_key(password, salt)

        # Desciframos el mensaje.
        try:
            decrypted_message = decrypt(encrypted_message, key)
            print(f"Successfully decrypted message: {decrypted_message}")
        except:
            print("Decryption failed. Incorrect password.")
    else:
        print("No encrypted message available for decryption.")

encrypt_password_input = widgets.Password(description="Encrypt Password:")
decrypt_password_input = widgets.Password(description="Decrypt Password:")
message_input = widgets.Textarea(description="Message:")
encrypt_button = widgets.Button(description="Encrypt")
decrypt_button = widgets.Button(description="Decrypt")
encrypted_message_output = widgets.Text(description="Encrypted message:", disabled=True)
salt_output = widgets.Text(description="Salt:", disabled=True, layout=widgets.Layout(display="none"))

encrypt_button.on_click(on_encrypt_button_click)
decrypt_button.on_click(on_decrypt_button_click)

In [ ]:
display(encrypt_password_input, message_input, encrypt_button) #prompteamos el input de contraseña y mensaje

Password(description='Encrypt Password:')

Textarea(value='Siuuuuu', description='Message:')

Button(description='Encrypt', style=ButtonStyle())

Encrypted message: b'gAAAAABkML7d7IsInsc6cJpahMVmNVNC7v5O_F5t8ieVcxAvxtbYmiMkH25NSbu8snN7Ueg562mYumo3Hic9DyRag0eXMlAlv1CALjscuPK_ANk9WeXIBqcqrmwG09hcASKPdbEEBF3d'


In [ ]:
display(decrypt_password_input, decrypt_button, encrypted_message_output, salt_output) #ahora el otro lado tiene que poner la llave simmetrica para pder descrifrar el mensaje.

Password(description='Decrypt Password:')

Button(description='Decrypt', style=ButtonStyle())

Text(value='gAAAAABkML63E-DUx_barYg6mk0mSUaVcQnW4lnCSyH9lCtWkA33sSAtbNg_hMQnJn4S0-XupVeHRedvvJ5rB_n8crDuR5o57Q…

Text(value='N8h4kfTo+4tpm83C/5MyAw==', description='Salt:', disabled=True, layout=Layout(display='none'))

Decryption failed. Incorrect password.
Successfully decrypted message: Ataquen la base militar de japon, orden 64
